# Analyze Jarvis Natural Language Classifications

## Harrison Myers

This notebook analyzes the output from the Jarvis chatbot natural language classifier deployed in Slack.

In [ ]:
import os
from zipfile import ZipFile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from src.helper_functions import create_dirs, load_model_results

### Result Exploration

In [ ]:
res = load_model_results("outputs/model_results.txt")

NB_accuracy = np.sum(res["NB_class"]) / len(res["NB_class"])
SGD_accuracy = np.sum(res["SGD_class"]) / len(res["SGD_class"])

# split into label dataframes
labels = ["TIME", "PIZZA", "WEATHER", "GREET", "JOKE"]

label_df_dict = dict.fromkeys(labels)

label_num_accurate_NB  = []
label_num_accurate_SGD = []
label_accuracy_NB = []
label_accuracy_SGD = []

for key, value in label_df_dict.items():
    label_df_dict[key] = res[res["Label"] == key]

for key, value in label_df_dict.items():
    label_num_accurate_NB.append(np.sum(value["NB_class"]))
    label_accuracy_NB.append(round(np.sum(value["NB_class"])/len(value["NB_class"]), 3))
    label_num_accurate_SGD.append(np.sum(value["SGD_class"]))
    label_accuracy_SGD.append(round(np.sum(value["SGD_class"])/len(value["SGD_class"]), 3))

# Append overall accuracy
labels.append("All Categories")
label_accuracy_NB.append(NB_accuracy)
label_accuracy_SGD.append(SGD_accuracy)

# Plot the data

fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.35  # Adjust the width of the bars

# Generating indexes for bars
x = np.arange(len(labels))

# Plot NB accuracy
ax.bar(x - bar_width/2, label_accuracy_NB, bar_width, label='Naive Bayes Accuracy', color='skyblue')

# Plot SGD accuracy
ax.bar(x + bar_width/2, label_accuracy_SGD, bar_width, label='SGD Accuracy', color='orange')

ax.set_xlabel('Labels')
ax.set_ylabel('Accuracy')
ax.set_title('NB vs.SGD model accuracy by Category')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
plt.tight_layout()
plt.show()